In [1]:
!nvidia-smi

Sun Mar 23 15:10:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:38:00.0 Off |                    0 |
| N/A   31C    P8             16W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/CLIP"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/ml-mobileclip"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/lvis-api"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git"

# !pip install -q supervision jupyter_bbox_widget

!wget -q https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_blt.pt

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/ubuntu/projects/football


In [3]:
from roboflow import Roboflow
# from google.colab import userdata
from IPython.display import Image

In [5]:
from dotenv import load_dotenv

env_path = f"{HOME}/.env"
load_dotenv(dotenv_path=env_path)
ROBOFLOW_API_KEY = os.environ.get("ROBOFLOW_API_KEY")


In [6]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(12)
dataset = version.download("yolov11")

loading Roboflow workspace...


loading Roboflow project...


In [7]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

In [8]:
import torch

# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    # Set default CUDA device
    torch.cuda.set_device(0)
    # Display memory information
    print(f"Total GPU memory: {torch.cuda.get_device_properties().total_memory / 1e9:.2f} GB")
    print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Reserved GPU memory: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
else:
    print("CUDA is not available. Training will use CPU only.")

CUDA available: True
CUDA device count: 4
CUDA device name: NVIDIA L4
CUDA version: 12.4
Total GPU memory: 23.57 GB
Allocated GPU memory: 0.00 GB
Reserved GPU memory: 0.00 GB


In [19]:
from huggingface_hub import hf_hub_download

path = hf_hub_download(repo_id="jameslahm/yoloe", filename="yoloe-11l-seg.pt", local_dir='.')
# path = hf_hub_download(repo_id="jameslahm/yoloe", filename="yoloe-11l-seg-pf.pt", local_dir='.')

yoloe-11l-seg.pt:   0%|          | 0.00/71.0M [00:00<?, ?B/s]

In [9]:
project_name = "app/models/yoloe"
lr0 = 1e-3

In [ ]:
%cd {HOME}

# from ultralytics import YOLOE
# from ultralytics.models.yolo.yoloe.train_pe import YOLOEPETrainer, YOLOEPESegTrainer
from ultralytics import YOLO
import os
from ultralytics.nn.tasks import guess_model_scale
from ultralytics.utils import yaml_load, LOGGER
import torch

os.environ["PYTHONHASHSEED"] = "0"  # Disable hash randomization

/home/ubuntu/projects/football


ImportError: cannot import name 'YOLOE' from 'ultralytics' (/home/ubuntu/miniconda3/envs/football/lib/python3.11/site-packages/ultralytics/__init__.py)

In [12]:
import os
HOME = os.getcwd()
print(HOME)

/home/ubuntu/projects/football


In [21]:
# !pip install -U ultralytics

In [22]:
# Load pre-trained YOLOE model and train on custom data
model = YOLO("yoloe-11l-seg.pt")
model.train(data=f"{dataset.location}/data.yaml",
            epochs=50, imgsz=1400, batch=16,
            device=[0,1,2,3], project="app/models/yoloe",
            name="yoloe_football", plots=True, lr0=lr0,
            )

AttributeError: Can't get attribute 'YOLOESegModel' on <module 'ultralytics.nn.tasks' from '/home/ubuntu/miniconda3/envs/football/lib/python3.11/site-packages/ultralytics/nn/tasks.py'>